In [6]:
import pandas as pd
import numpy as np
import os
import datetime

import getpass

usuario = getpass.getuser()



In [7]:
usuario

'lravlic'

LECTURA DE DATAFRAMES

In [8]:

#Lectura de los archivos de la ME2L en el repositorio de Sharepoint
me2l1_df = pd.read_excel(f"C:/Users/{usuario}/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/AFM/ME2L1.XLSX", dtype={'Documento compras':'str','Posición':'str', 'Material':'str'})
me2l2_df = pd.read_excel(f"C:/Users/{usuario}/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/AFM/ME2L2.XLSX", dtype={'Documento compras':'str','Posición':'str', 'Material':'str'})
me2l3_df = pd.read_excel(f"C:/Users/{usuario}/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/AFM/ME2L3.XLSX", dtype={'Documento compras':'str','Posición':'str', 'Material':'str'})

#eliminacion de filas en blanco que vienen por defecto en SAP
me2l1_df = me2l1_df.drop(0)
me2l2_df = me2l2_df.drop(0)
me2l3_df = me2l3_df.drop(0)


#consolidacion de todos los dataframes de me2l en uno solo
me2l_consolidado = pd.concat([me2l1_df,me2l2_df,me2l3_df])
me2l_consolidado.reset_index(drop=True, inplace=True)

In [9]:
datetime.datetime.today().year

2024

In [10]:
#localizacion y lectura de base planificable 

base = f"C:/Users/{usuario}/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros/Base Planificable"
base_carpeta = os.listdir(base)
for i in base_carpeta:
    if str(datetime.datetime.today().year) + '-' + str(datetime.datetime.today().month).zfill(2) in i:
        ruta = os.path.join(base, i)
        print(ruta)

        ruta_base = os.listdir(ruta)
        for i in ruta_base:
            if 'Base' in i and not 'R3' in i:
                ruta_final = os.path.join(ruta, i)
                print(ruta_final)
                base = pd.read_excel(ruta_final,header=1)
                base_df = base[['Material','Texto breve de material','NomSector_actual','TIPO','Corresponde']]
                base_df_prov = base[['Proveedor', 'Pais (Proveedor)']]
        
base_df_prov = base_df_prov.drop_duplicates(subset=['Proveedor'])

C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros/Base Planificable\2024-04 Base Planificable
C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros/Base Planificable\2024-04 Base Planificable\Base Abril.xlsx


In [11]:
ruta = f"C:/Users/{usuario}/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros"
lista_ruta = os.listdir(ruta)
for i in lista_ruta:
    if str(datetime.datetime.today().year) in i:
        ruta = os.path.join(ruta , i)
        lista_ruta = os.listdir(ruta)
        for i in lista_ruta:
            
            if str(datetime.datetime.today().month).zfill(2) in i:
                ruta = os.path.join(ruta, i)
                lista_ruta = os.listdir(ruta)
                for i in lista_ruta:
                    if 'COD_ACTUAL_S4' in i and not 'R3' in i:
                        ruta = os.path.join(ruta, i)
                        print(ruta)
                        cod_actual_df = pd.read_excel(ruta, usecols = ['Nro_pieza_fabricante_1', 'Cod_Actual_1'])

C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros\2024\2024-04\COD_ACTUAL_S4_20240331.xlsx


In [12]:
base_df_ue = pd.merge(base_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")

In [13]:
base_df_ue['Cod_Actual_1'] = base_df_ue['Cod_Actual_1'].fillna(base_df_ue['Material'])

In [14]:
base_df_ue = base_df_ue[['Cod_Actual_1','Texto breve de material', 'NomSector_actual', 'TIPO','Corresponde']]
base_df_ue = base_df_ue.rename(columns={'Cod_Actual_1':'Material'})

In [15]:
base_df_ue = base_df_ue.drop_duplicates(subset=['Material'])

In [16]:
base_df_prov = base_df_prov.drop_duplicates(subset=['Proveedor'])

TRATAMIENTO DE DFS (OC, POSiCION)

In [17]:
print(f'Dimensiones del df1: {me2l1_df.shape}')
print('-' * 40)
print(f'Dimensiones del df2: {me2l2_df.shape}')
print('-' * 40)
print(f'Dimensiones del df3: {me2l3_df.shape}')


Dimensiones del df1: (2892, 39)
----------------------------------------
Dimensiones del df2: (4542, 39)
----------------------------------------
Dimensiones del df3: (3869, 39)


UNIR DFS

In [18]:
me2l_consolidado = pd.concat([me2l1_df,me2l2_df,me2l3_df])


In [19]:
me2l_consolidado.reset_index(drop=True, inplace=True)

In [20]:
me2l_consolidado['Material'] = me2l_consolidado['Material'].astype('str')
me2l_consolidado['Material'] = me2l_consolidado['Material'].apply(lambda x: x.split(".")[0])

In [21]:
#me2l_consolidado.drop(columns=['Reparto','Tipo doc.compras','Historial pedido/Docu.orden entrega','Grupo de artículos','Tipo de posición', 'Tipo de imputación', 'Cantidad de reparto','Hora'], inplace=True)

In [22]:
print(f'Dimensiones de consolidado: {me2l_consolidado.shape}')

Dimensiones de consolidado: (11303, 39)


ULTIMO ESLABON A ME2L

In [23]:
me2l_ue = pd.merge(me2l_consolidado, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")

In [24]:
me2l_ue['Cod_Actual_1'].fillna(me2l_ue['Material'], inplace=True)

In [26]:
me2l_ue = me2l_ue[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material','Cod_Actual_1', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega','Hora', 'Fecha entrega estad.', 'Cantidad anterior',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre de proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)'
       ]]


In [27]:
me2l_ue = me2l_ue.rename(columns={'Material':'Material Antiguo'})
me2l_ue = me2l_ue.rename(columns={'Cod_Actual_1':'Material'})


In [28]:
datetime.datetime.today().strftime('%Y-%m-%d')

'2024-04-16'

In [28]:
me2l_ue.to_excel(f"C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Tubo Semanal/{datetime.datetime.today().strftime('%Y-%m-%d')}/{datetime.datetime.today().strftime('%Y-%m-%d')} ME2L S4.xlsx")

In [29]:
me2l_cruce_sector = pd.merge(me2l_ue, base_df_ue[['Material','Texto breve de material', 'NomSector_actual', 'TIPO','Corresponde']], left_on="Material", right_on="Material", how="left")

In [30]:
me2l_cruce_sector.shape

(11303, 43)

In [31]:
me2l_cruce_sector = me2l_cruce_sector[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega','Hora', 'Fecha entrega estad.', 'Cantidad anterior',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre de proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual',
       'Texto breve de material', 'TIPO', 'Corresponde']]

In [32]:
me2l_cruce_sector['Cod_Prov'] = me2l_cruce_sector['Nombre de proveedor'].str.split(' ', expand=True)[0]

In [33]:
me2l_cruce_sector = me2l_cruce_sector.merge(base_df_prov,left_on='Cod_Prov', right_on='Proveedor', how='left')

In [35]:
me2l_cruce_sector['Posición'] = me2l_cruce_sector['Posición'].astype('str')

In [36]:
me2l_cruce_sector['AUX'] = me2l_cruce_sector['Documento compras'] + me2l_cruce_sector['Posición']

In [37]:
me2l_cruce_sector['Origen'] = me2l_cruce_sector['Pais (Proveedor)']

In [38]:
me2l_cruce_sector.columns

Index(['Documento compras', 'Posición', 'Reparto', 'Cl.documento compras',
       'Tipo doc.compras', 'Grupo de compras',
       'Historial pedido/Docu.orden entrega', 'Fecha documento',
       'Material Antiguo', 'Material', 'Texto breve', 'Grupo de artículos',
       'Indicador de borrado', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega', 'Hora', 'Fecha entrega estad.', 'Cantidad anterior',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre de proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual',
       'Texto breve de material', 'TIPO', 'Corresponde', 'Cod_Prov',
       'Proveedor', 'Pais (Pro

In [39]:
me2l_cruce_sector = me2l_cruce_sector[['AUX','Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega','Hora', 'Fecha entrega estad.', 'Cantidad anterior',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre de proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual','Origen', 'TIPO',  'Corresponde']]

In [40]:
me2l_cruce_sector.shape

(11303, 44)

In [41]:
me2l_base_tr = me2l_cruce_sector[me2l_cruce_sector['NomSector_actual'].notna()]

In [42]:
me2l_base_tr['Corresponde'].fillna(0, inplace=True)
me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)

C:\Users\lravlic\AppData\Local\Temp\ipykernel_23932\3910453820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].fillna(0, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_23932\3910453820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)


In [43]:
bases_oc = me2l_base_tr.groupby(['Documento compras'])['Posición'].count().sort_values(ascending=False).reset_index()

In [44]:
me2l_base_tr['Origen'].value_counts()

Origen
CN    2464
CL    1398
JP    1334
TR     767
BR     493
KR     427
US     340
IN     338
IT     247
DE     211
GB     210
MX     197
AR     172
ID     107
HU      54
FR      47
TW      44
MY      24
NL      22
CO      19
PE      18
TH      15
HK       8
ES       3
Name: count, dtype: int64

In [104]:


# Calculate the number of rows in each part
num_rows_per_part = len(bases_oc) // 5  # Integer division to get an equal split

# Split the DataFrame into five parts
df_parts = np.array_split(bases_oc, 5)


# Alternatively, you can access the parts using indexing (e.g., df_parts[0], df_parts[1], etc.)

# Each part will have a similar number of rows


c:\Users\lravlic\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [52]:
df_parts[0]['Documento compras'].to_clipboard(header=False, index=False)

In [53]:
df_parts[1]['Documento compras'].to_clipboard(header=False, index=False)

In [54]:
df_parts[2]['Documento compras'].to_clipboard(header=False, index=False)

In [55]:
df_parts[3]['Documento compras'].to_clipboard(header=False, index=False)

In [56]:
df_parts[4]['Documento compras'].to_clipboard(header=False, index=False)

In [67]:
bases_oc['Documento compras'].to_clipboard(header=False, index=False)

In [91]:
lista = os.listdir('C:/Users/lravlic/PROYECTOS DATA/PRUEBAS TRANSITO')

In [92]:
dfs = []

for i in lista:
    if 'MONITOR' in i or 'monitor' in i:
        df = pd.read_excel(i, index_col=None)
        dfs.append(df)


In [95]:
len(dfs)

3

In [93]:
for i in lista:
    if 'ee' in i or 'EE' in i or 'entrega entrante' in i or 'ENTREGA ENTRANTE' in i:
        df_ee = pd.read_excel(i)
        print(i)

EE.XLSX


In [94]:
monitor_df = pd.concat(dfs,axis=0, ignore_index=True)

In [96]:
monitor_df.shape

(134106, 106)

In [72]:
#monitor_df = monitor_df[['Nro. OC SAP','Clase Doc. OC','Posición OC SAP','Ind. Borrado OC','Clase Doc. SP','Nro. SOLPED','Posición SOLPED','Ind. Borrado SP','Cant. UN. OC','N° MIRO','Posición Factura',
#'Cant. Factura','Fe. Confirmación','Fe. Compromiso','Fe. ATA','TF de ATA','Cantidad en BO','Ind. Borrado PO']]

In [97]:
monitor_df['Nro. OC SAP'] = monitor_df['Nro. OC SAP'].astype('str')
monitor_df['Posición OC SAP'] = monitor_df['Posición OC SAP'].astype('str')


In [98]:
monitor_df['AUX'] = monitor_df['Nro. OC SAP'] + monitor_df['Posición OC SAP']

In [99]:
monitor_df['N° MIRO'] = monitor_df['N° MIRO'].astype('str')

In [100]:
#revision = monitor_df.groupby('AUX')['Fe. ATA'].nunique().sort_values(ascending=False).reset_index()

In [101]:
tabla_1 = monitor_df[monitor_df['Ind. Borrado OC'].isna() &
    monitor_df['Ind. Borrado SP'].isna() &
    monitor_df['Ind. Borrado PO'].isna()].groupby(['AUX', 'N° MIRO']).agg({'Cant. UN. OC':'mean', 'Cant. Factura':'max','Cantidad en BO':'mean'})

In [102]:
tabla_1

,,Cant. UN. OC,Cant. Factura,Cantidad en BO
AUX,N° MIRO,,,
420000203610,5105776555.0,6.0,6,0.0
4200002036100,5105780925.0,1.0,1,0.0
42000020361000,5105828213.0,8.0,8,0.0
42000020361010,5105780925.0,1.0,1,0.0
42000020361020,5105776555.0,1.0,1,0.0
...,...,...,...,...
470002223450,nan,191.0,0,191.0
470002223460,nan,192.0,0,192.0
470002223470,nan,93.0,0,93.0


In [103]:
tabla_final = tabla_1.groupby(['AUX']).agg({'Cant. UN. OC':'mean','Cant. Factura':'sum', 'Cantidad en BO':'mean'}).reset_index()

In [104]:
print(f'Dimension de tabla final: {tabla_final.shape}')
print('-'*40)
print(f'Total Campo OC: {tabla_final['Cant. UN. OC'].sum()}')
print(f'Total Campo Factura: {tabla_final['Cant. Factura'].sum()}')
print(f'Total Campo BO: {tabla_final['Cantidad en BO'].sum()}')

Dimension de tabla final: (86342, 4)
----------------------------------------
Total Campo OC: 3736234.0
Total Campo Factura: 1865314
Total Campo BO: 1855168.0


In [106]:
df_ee = df_ee[['Documento compras','Posición modelo', 'Cantidad entrega', 'Estado movim.mcías.']]

In [107]:
df_ee['Documento compras'] = df_ee['Documento compras'].astype('str')
df_ee['Posición modelo'] = df_ee['Posición modelo'].astype('str')

In [108]:
df_ee.dtypes

Documento compras       object
Posición modelo         object
Cantidad entrega       float64
Estado movim.mcías.     object
dtype: object

In [109]:
df_ee['AUX'] = df_ee['Documento compras'] + df_ee['Posición modelo']
df_ee.drop(columns=['Documento compras','Posición modelo'])

,Cantidad entrega,Estado movim.mcías.,AUX
0,1.0,C,430000110030
1,1.0,C,4300001100210
2,1.0,C,4300001277100
3,1.0,C,4300001277200
4,1.0,C,430000127830
...,...,...,...
155947,74.0,A,470002198950
155948,40.0,A,470002198960
155949,33.0,A,470002198970
155950,29.0,A,470002198980


In [110]:
table_ee = df_ee[df_ee['Estado movim.mcías.']=='C'].groupby(['AUX'])['Cantidad entrega'].sum().reset_index()

In [111]:
table_ee[table_ee['AUX']=='400002095760']

,AUX,Cantidad entrega
0,400002095760,1.0


In [88]:
table_ee.to_excel('Entrega Entrante TD.xlsx')

KeyboardInterrupt: 

In [86]:
me2l_base_tr.to_excel('Base_+_15.xlsx')

In [89]:
me2l_base_tr.shape

(17920, 44)

In [112]:
me2l_base_tr['Origen'] =  np.where(me2l_base_tr['Origen'] == 'CL', 'NAC', 'IMP')

C:\Users\lravlic\AppData\Local\Temp\ipykernel_23932\869764248.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Origen'] =  np.where(me2l_base_tr['Origen'] == 'CL', 'NAC', 'IMP')


In [113]:
me2l_base_tr['Origen'].value_counts()

Origen
IMP    11182
Name: count, dtype: int64

In [114]:
me2l_15 = me2l_base_tr

In [115]:
monitor_df.drop_duplicates(subset=['AUX'], inplace=True)

In [116]:
me2l_15 = me2l_base_tr

In [117]:
monitor_df['Fe. ATA'] = pd.to_datetime(monitor_df['Fe. ATA'])

In [118]:
me2l_15 = me2l_15.merge(monitor_df[['AUX','Fe. ATA']], left_on='AUX', right_on='AUX', how = 'left')

In [85]:
me2l_15['Fe. ATA'].head(30)

0     NaT
1     NaT
2     NaT
3     NaT
4     NaT
5     NaT
6     NaT
7     NaT
8     NaT
9     NaT
10    NaT
11    NaT
12    NaT
13    NaT
14    NaT
15    NaT
16    NaT
17    NaT
18    NaT
19    NaT
20    NaT
21    NaT
22    NaT
23    NaT
24    NaT
25    NaT
26    NaT
27    NaT
28    NaT
29    NaT
Name: Fe. ATA, dtype: object

In [119]:
me2l_15['Fecha Est. Fact'] = np.where(me2l_15['Fe. ATA'].notnull(), me2l_15['Fe. ATA'] + pd.Timedelta(days=15), me2l_15['Fecha de entrega'])

In [120]:
tabla_final.head()

,AUX,Cant. UN. OC,Cant. Factura,Cantidad en BO
0,420000203610,6.0,6,0.0
1,4200002036100,1.0,1,0.0
2,42000020361000,8.0,8,0.0
3,42000020361010,1.0,1,0.0
4,42000020361020,1.0,1,0.0


In [121]:
me2l_15 = me2l_15.merge(tabla_final[['AUX', 'Cant. Factura']], left_on = 'AUX', right_on='AUX',how='left')

In [122]:
me2l_15['Cant. Factura'] = me2l_15['Cant. Factura'] * me2l_15['Corresponde']

In [123]:
me2l_15= me2l_15.merge(table_ee, left_on='AUX', right_on='AUX', how='left')

In [124]:
me2l_15['Cantidad entrega'] = me2l_15['Cantidad entrega']*me2l_15['Corresponde']

In [125]:
me2l_15['Cantidad entrega'].sum()

83439.99799999999

In [126]:
me2l_15['Qty por Entregar'] = me2l_15['Por entregar (cantidad)']*me2l_15['Corresponde']

In [127]:
me2l_15['Qty por Entregar'].sum()

3032365.0

In [128]:
#cambiar esto por la fecha del dia actual

hoy = datetime.datetime.today()

In [129]:
me2l_15['Cantidad entrega'].fillna(0, inplace=True)

In [130]:


# Suponiendo que 'me2l_15' es tu DataFrame y 'AW4', 'BJ4' y 'AT4' son columnas en tu DataFrame
result = me2l_15['Cant. Factura'] - me2l_15['Cantidad entrega']
result = np.where(np.isnan(result), me2l_15['Qty por Entregar'], result)

me2l_15['Qty Fact Corr'] = np.where(result<0, 0, result)

In [131]:
me2l_15['Qty Fact Corr'].sum()

600280.0

In [132]:
hoy = pd.to_datetime(hoy)

In [133]:
me2l_15[ 'Fecha Est. Fact'].max()

Timestamp('2024-12-24 00:00:00')

In [134]:
conditions = [
    (me2l_15['Qty Fact Corr'] > 0) & (me2l_15['Fecha Est. Fact'] >= hoy),
    (me2l_15['Qty Fact Corr'] > 0) & (me2l_15['Fecha Est. Fact'] < hoy)
]

choices = ['Facturado No Vencido', 'Facturado Vencido']

me2l_15['Status TR Fact'] = np.select(conditions, choices, default='No Facturado')

In [135]:
me2l_15['Status TR Fact'].value_counts()

Status TR Fact
No Facturado            9315
Facturado No Vencido    1568
Facturado Vencido        299
Name: count, dtype: int64

In [136]:
from datetime import timedelta, datetime

hoy_31 = hoy  + timedelta(days=31)

In [137]:
hoy_31 = pd.to_datetime(hoy_31)

In [138]:
condition = (me2l_15['Fecha de entrega'] > hoy) & (me2l_15['Fecha de entrega'] < hoy_31)

# Apply the condition using boolean indexing and assign 'SI' or 'NO' accordingly
me2l_15['Fecha BO Mes Actual'] = np.where(condition, 'SI', 'NO')

In [139]:
me2l_15['Fecha BO Mes Actual'].value_counts()

Fecha BO Mes Actual
NO    9604
SI    1578
Name: count, dtype: int64

In [140]:
condition= (me2l_15['Fecha de entrega']< hoy)

me2l_15['TR Vencido'] = np.where(condition, 'SI','NO')

In [141]:
me2l_15['TR Vencido'].value_counts()

TR Vencido
NO    6299
SI    4883
Name: count, dtype: int64

In [142]:
conditions = [
    (me2l_15['Origen'] == 'NAC') & (me2l_15['TIPO']=='AFM'),
    (me2l_15['Fecha BO Mes Actual'] == "NO") & (me2l_15['TR Vencido'] == "NO"),
    (me2l_15['TR Vencido'] == "SI"),
]

# Define choices
choices = [
    "Replanificar +45",
    "Fecha Teórica",
    "TR Vencido",
]

# Apply np.select
me2l_15['Statur TR No FAct'] = np.select(conditions, choices, default="Replanificar +45")

In [143]:
me2l_15['Cant. Factura'].sum()

683277.0

In [144]:
#Qty BO
me2l_15['Cant. BO'] = me2l_15['Cant. Factura'].fillna(0, inplace=True)

In [124]:
#Cant Factura : Cant Fact CORR
#QTY BO: CHECK

In [145]:
condicion = (me2l_15['Fecha Est. Fact'] < hoy)

me2l_15['Atraso'] = np.where(condicion, 'SI','NO')

In [146]:
me2l_15['Atraso'].value_counts()

Atraso
NO    6313
SI    4869
Name: count, dtype: int64

In [147]:
me2l_15['Fecha Est. Fact']

0       2024-05-09
1       2024-05-09
2       2024-05-09
3       2024-05-09
4       2024-05-09
           ...    
11177   2022-11-30
11178   2022-11-30
11179   2022-11-30
11180   2022-11-30
11181   2022-11-30
Name: Fecha Est. Fact, Length: 11182, dtype: datetime64[ns]

In [148]:
condicion = (me2l_15['Atraso'] == 'SI')

me2l_15['Fecha Facturacion Final'] = np.where(condicion, hoy_31, me2l_15['Fecha Est. Fact'])



In [149]:
me2l_15['Fecha Facturacion Final']

0               1715212800000000000
1               1715212800000000000
2               1715212800000000000
3               1715212800000000000
4               1715212800000000000
                    ...            
11177    2024-05-17 11:39:21.734456
11178    2024-05-17 11:39:21.734456
11179    2024-05-17 11:39:21.734456
11180    2024-05-17 11:39:21.734456
11181    2024-05-17 11:39:21.734456
Name: Fecha Facturacion Final, Length: 11182, dtype: object

In [150]:
me2l_15['Fecha Facturacion Final'] = pd.to_datetime(me2l_15['Fecha Facturacion Final'])

In [151]:
me2l_15['Fecha Facturacion Final']

0       2024-05-09 00:00:00.000000
1       2024-05-09 00:00:00.000000
2       2024-05-09 00:00:00.000000
3       2024-05-09 00:00:00.000000
4       2024-05-09 00:00:00.000000
                   ...            
11177   2024-05-17 11:39:21.734456
11178   2024-05-17 11:39:21.734456
11179   2024-05-17 11:39:21.734456
11180   2024-05-17 11:39:21.734456
11181   2024-05-17 11:39:21.734456
Name: Fecha Facturacion Final, Length: 11182, dtype: datetime64[ns]

In [152]:
monitor_df.dtypes

Prefijo OC              object
Nro. OC SAP             object
Clase Doc. OC           object
Posición OC SAP         object
Ind. Borrado OC         object
                         ...  
Cantidad en BO           int64
Pet.Ofert.Creado por    object
OC Creado por           object
Marcar para "Cerrar"    object
AUX                     object
Length: 107, dtype: object

In [153]:
me2l_15 = me2l_15.merge(monitor_df[['AUX', 'Vía (Texto)']], left_on='AUX', right_on='AUX', how='left')

In [154]:
me2l_15['Vía (Texto)'].value_counts()

Vía (Texto)
Marítimo     5879
Aéreo         715
Terrestre     517
Courier       419
Name: count, dtype: int64

In [155]:
me2l_15.columns

Index(['AUX', 'Documento compras', 'Posición', 'Reparto',
       'Cl.documento compras', 'Tipo doc.compras', 'Grupo de compras',
       'Historial pedido/Docu.orden entrega', 'Fecha documento',
       'Material Antiguo', 'Material', 'Texto breve', 'Grupo de artículos',
       'Indicador de borrado', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega', 'Hora', 'Fecha entrega estad.', 'Cantidad anterior',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre de proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual',
       'Origen', 'TIPO', 'Corresponde', 'Fe. ATA', 'Fecha Est. Fact',
       'Cant. Factura'

In [156]:
me2l_15['LT Objetivo'] = np.where(me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"]), 45,
                    np.where(me2l_15['Vía (Texto)'] == "Aéreo", 15,
                        np.where(me2l_15['Vía (Texto)'] == "Courier", 7, 0)))

In [157]:
me2l_15 = me2l_15.merge(monitor_df[['AUX','Fe. Confirmación']], left_on='AUX',right_on='AUX', how='left')

In [158]:
me2l_15['Fecha OC'] = np.where((me2l_15['Fe. Confirmación'] == 0) | (me2l_15['Fe. Confirmación'].isna()), me2l_15['Fecha documento'], me2l_15['Fe. Confirmación'])

In [159]:
import datetime
today = hoy

In [160]:
today = pd.to_datetime(today)

In [161]:


# Calculate the difference
me2l_15['Dias LT BO'] = (today - me2l_15['Fecha OC']).dt.days - me2l_15['LT Objetivo']

# Handling potential errors, setting them to 0
me2l_15['Dias LT BO'] = me2l_15['Dias LT BO'].fillna(0)

In [162]:
me2l_15['Dias LT BO'].sum()

4515670

In [163]:
import pandas as pd
import numpy as np

# Define conditions and choices
conditions = [
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(1, 30)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(31, 60)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(61, 90)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'] > 90),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(1, 15)),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(16, 30)),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(31, 45)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(1, 7)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(8, 15)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(16, 21)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'] > 21),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'] > 45)
]

choices = [
    "De 1 a 30 días",
    "De 31 a 60 días",
    "De 61 a 90 días",
    "De 91 días a más",
    "De 1 a 15 días",
    "De 16 a 30 días",
    "De 31 a 45 días",
    "De 1 a 7 días",
    "De 8 a 15 días",
    "De 16 a 21 días",
    "De 21 días a más",
    "De 45 días a más"
]

# Assign the values based on conditions
me2l_15['Categoría LT'] = np.select(conditions, choices, default="Dentro de LT")



In [164]:
me2l_15['Categoría LT'].value_counts()

Categoría LT
Dentro de LT        5283
De 1 a 30 días      1596
De 91 días a más    1482
De 31 a 60 días     1194
De 45 días a más     715
De 61 a 90 días      493
De 21 días a más     419
Name: count, dtype: int64

In [165]:
import pandas as pd
import numpy as np

# Assuming CE4 is a column in the DataFrame and BV1 is a variable
BV1 = 45  # Assuming the value of $BV$1 is 45

# Define conditions and choices
conditions = [
    me2l_15['Statur TR No FAct'] == "Replanificar +45",
    me2l_15['Statur TR No FAct'] == "Fecha Teórica",
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & ((me2l_15['Categoría LT'] == 0) | (me2l_15['Categoría LT'] == "Dentro de LT")),
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & (me2l_15['Categoría LT'] == "De 31 a 60 días"),
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & (me2l_15['Categoría LT'] == "De 61 a 90 días")
]

choices = [
    hoy + timedelta(days =45),
    me2l_15['Fecha Facturacion Final'],
    hoy + timedelta(days =30),
    hoy + timedelta(days =60),
    hoy + timedelta(days =90)
]

# Assign the values based on conditions
me2l_15['Fecha No Fact'] = np.select(conditions, choices, default=hoy + timedelta(days =120))


In [166]:
me2l_15['Fecha No Fact'] = pd.to_datetime(me2l_15['Fecha No Fact'])

In [167]:
tabla_final.columns

Index(['AUX', 'Cant. UN. OC', 'Cant. Factura', 'Cantidad en BO'], dtype='object')

In [168]:
me2l_15= me2l_15.merge(tabla_final[['AUX','Cantidad en BO']], left_on='AUX',right_on='AUX', how='left')

In [169]:
me2l_15['Cantidad en BO'] = me2l_15['Cantidad en BO']*me2l_15['Corresponde']

In [170]:
me2l_15['Cantidad en BO'].fillna(0, inplace=True)

In [171]:
me2l_15.columns

Index(['AUX', 'Documento compras', 'Posición', 'Reparto',
       'Cl.documento compras', 'Tipo doc.compras', 'Grupo de compras',
       'Historial pedido/Docu.orden entrega', 'Fecha documento',
       'Material Antiguo', 'Material', 'Texto breve', 'Grupo de artículos',
       'Indicador de borrado', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega', 'Hora', 'Fecha entrega estad.', 'Cantidad anterior',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre de proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual',
       'Origen', 'TIPO', 'Corresponde', 'Fe. ATA', 'Fecha Est. Fact',
       'Cant. Factura'

In [172]:
import numpy as np

# Crear una condición basada en la columna 'TIPO'
condicion = (me2l_15['TIPO'] == 'OEM') & (me2l_15['Por calcular (cantidad)'] == 0)

# Actualizar la columna 'Cantidad en BO' basada en la condición
me2l_15['Cantidad en BO'] = np.where(condicion, 0, me2l_15['Cantidad en BO'])



In [173]:
me2l_15['TIPO'].value_counts()

TIPO
AFM    6457
OEM    4657
AXS      68
Name: count, dtype: int64

In [174]:
me2l_15['Cantidad en BO'].sum()

2421454.0

In [175]:
me2l_15[me2l_15['AUX'] =='430002269010']

,AUX,Documento compras,Posición,Reparto,Cl.documento compras,Tipo doc.compras,Grupo de compras,Historial pedido/Docu.orden entrega,Fecha documento,Material Antiguo,...,Atraso,Fecha Facturacion Final,Vía (Texto),LT Objetivo,Fe. Confirmación,Fecha OC,Dias LT BO,Categoría LT,Fecha No Fact,Cantidad en BO


In [176]:
me2l_15.columns

Index(['AUX', 'Documento compras', 'Posición', 'Reparto',
       'Cl.documento compras', 'Tipo doc.compras', 'Grupo de compras',
       'Historial pedido/Docu.orden entrega', 'Fecha documento',
       'Material Antiguo', 'Material', 'Texto breve', 'Grupo de artículos',
       'Indicador de borrado', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega', 'Hora', 'Fecha entrega estad.', 'Cantidad anterior',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre de proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual',
       'Origen', 'TIPO', 'Corresponde', 'Fe. ATA', 'Fecha Est. Fact',
       'Cant. Factura'

In [177]:
side_a = me2l_15[['AUX','Status TR Fact',   'Material','Texto breve','Qty Fact Corr','Centro','NomSector_actual','Origen','TIPO','Cl.documento compras','Fecha Facturacion Final','Nombre de proveedor','Grupo de compras']]
side_b = me2l_15[['AUX','Statur TR No FAct','Material','Texto breve','Cantidad en BO',     'Centro','NomSector_actual','Origen','TIPO','Cl.documento compras','Fecha No Fact',          'Nombre de proveedor','Grupo de compras']]

In [178]:
side_a.rename(columns={'Status TR Fact':'Status','Qty Fact Corr':'Cantidad','Fecha Facturacion Final':'Fecha'}, inplace=True)
side_b.rename(columns={'Statur TR No FAct':'Status','Cantidad en BO':'Cantidad','Fecha No Fact':'Fecha'}, inplace=True)

C:\Users\lravlic\AppData\Local\Temp\ipykernel_23932\410670440.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_a.rename(columns={'Status TR Fact':'Status','Qty Fact Corr':'Cantidad','Fecha Facturacion Final':'Fecha'}, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_23932\410670440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_b.rename(columns={'Statur TR No FAct':'Status','Cantidad en BO':'Cantidad','Fecha No Fact':'Fecha'}, inplace=True)


In [179]:
tr_final = pd.concat([side_a, side_b])

In [180]:
tr_final.reset_index()

,index,AUX,Status,Material,Texto breve,Cantidad,Centro,NomSector_actual,Origen,TIPO,Cl.documento compras,Fecha,Nombre de proveedor,Grupo de compras
0,0,420000714110,No Facturado,1067930,COOLANT AZUL -16 TROOP CJ 6B 1GL,0.0,714.0,Lubricantes,IMP,AFM,ZSTO,2024-05-09 00:00:00.000000,11460 COMERCIAL SURCHILE SPA,TT3
1,1,420000714120,No Facturado,1136080,LIMPIA VIDRIO 500ML FW-V0501 FRESHWAVE,0.0,714.0,Accesorios-Car Care,IMP,AFM,ZSTO,2024-05-09 00:00:00.000000,11460 COMERCIAL SURCHILE SPA,TT3
2,2,420000714130,No Facturado,1136082,PROTEC BRILLT 500ML FW-P0502 FRESHWAVE,0.0,714.0,Accesorios-Car Care,IMP,AFM,ZSTO,2024-05-09 00:00:00.000000,11460 COMERCIAL SURCHILE SPA,TT3
3,3,420000714140,No Facturado,115576,ANT/REF TROOP BAJO PUNTO CJ 6B1GL,0.0,714.0,Lubricantes,IMP,AFM,ZSTO,2024-05-09 00:00:00.000000,11460 COMERCIAL SURCHILE SPA,TT3
4,4,420000714150,No Facturado,116154,ANT/REF TROOP ROSADO -16° CJ 6B1GL,0.0,714.0,Lubricantes,IMP,AFM,ZSTO,2024-05-09 00:00:00.000000,11460 COMERCIAL SURCHILE SPA,TT3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22359,11177,4300017999390,TR Vencido,1023995,MANGUERA TOMA AIRE,0.0,710.0,Mazda,IMP,OEM,ZVOR,2024-08-14 11:39:21.734456,PE42 DERCO PERU S.A.,RR9
22360,11178,430001805210,TR Vencido,580940,BRAZO SUSP INF DEL IZQ,0.0,710.0,Mazda,IMP,OEM,ZVOR,2024-08-14 11:39:21.734456,PE42 DERCO PERU S.A.,RR9
22361,11179,4300018052110,TR Vencido,1023995,MANGUERA TOMA AIRE,0.0,710.0,Mazda,IMP,OEM,ZVOR,2024-08-14 11:39:21.734456,PE42 DERCO PERU S.A.,RR9
22362,11180,4300018052170,TR Vencido,511299,PARASOL,0.0,710.0,Mazda,IMP,OEM,ZVOR,2024-08-14 11:39:21.734456,PE42 DERCO PERU S.A.,RR9


In [181]:
tr_final['Cantidad'].sum()

3021734.0

In [182]:


# Suponiendo que tienes un DataFrame llamado df con una columna llamada 'fecha' que contiene fechas
# Si la columna 'fecha' no está en formato datetime, conviértela primero
tr_final['Fecha'] = pd.to_datetime(tr_final['Fecha'])

# Crear una nueva columna 'semana' que contenga el número de semana correspondiente a cada Fecha
tr_final['Semana'] = tr_final['Fecha'].dt.isocalendar().week

In [183]:


# Crear una nueva columna 'semana' que contenga el número de semana correspondiente a cada Fecha
tr_final['Mes'] = tr_final['Fecha'].dt.month

In [184]:


# Crear una nueva columna 'semana' que contenga el número de semana correspondiente a cada Fecha
tr_final['Año'] = tr_final['Fecha'].dt.year

In [185]:
tr_final.to_excel(f"C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Tubo Semanal/2024-04-15/2024-04-15 TR FINAL S4.xlsx")